In [161]:
import numpy as np
import pandas as pd
import sqlite3
import json
from os import getcwd, environ
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [162]:
environ["SPOTIPY_CLIENT_ID"]='35eb9288550941109f8e0582bcd6bdf3'
environ["SPOTIPY_CLIENT_SECRET"]='dc00c8b7e55140518d8f62954af8876f'

In [163]:
con = sqlite3.connect('track_metadata.db')
track_map = pd.read_sql_query("SELECT * FROM track_map", con)
con.close()
track_map.set_index('track_id',inplace=True)

In [164]:
print(track_map.head())

                               song_id
track_id                              
TRAAAAW128F429D538  SOMZWCG12A8C13C480
TRAAABD128F429CF47  SOCIWDW12A8C13D406
TRAAADZ128F9348C2E  SOXVLOJ12AB0189215
TRAAAEF128F4273421  SONHOTT12A8C13493C
TRAAAFD128F92F423A  SOFSOCN12A8C143F5D


In [165]:
spotify_feature_list = ['tempo','time_signature',\
       'danceability','energy','key','loudness','speechiness','acousticness','instrumentalness','valence','liveness']

In [166]:
def get_spotify_track_id(msd_track_id):
    song_code = track_map.loc[msd_track_id]['song_id']
    prefix_code = song_code[2:4]
    json_file_path = getcwd()+'/millionsongdataset_echonest/'+prefix_code+'/'+song_code+'.json'
    spotify_track_id = None
    with open(json_file_path,'r') as open_file:
        data = json.load(open_file)['response']['songs']
        if len(data) != 0:
            track_maps = data[0]['tracks']
            for mapping in track_maps:
                if mapping['catalog']=='spotify':
                    spotify_track_id = mapping['foreign_id']
                else: pass
    return spotify_track_id

In [167]:
def extract_df_row(spotify_track_id):
    spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
    track_data = spotify.audio_features([spotify_track_id])[0]
    return [track_data[feature] for feature in spotify_feature_list]

In [168]:
spotify_df = pd.DataFrame(columns=['track_id','tempo','time_signature',\
       'danceability','energy','key','loudness','speechiness','acousticness','instrumentalness','valence','liveness'])
spotify_df.set_index('track_id',inplace=True)

In [169]:
print(spotify_df.head())

Empty DataFrame
Columns: [tempo, time_signature, danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, valence, liveness]
Index: []


In [171]:
indices = track_map.index.values.tolist()
for count, track_id in enumerate(indices):
    if count%500==0:print(count)
    spotify_track_id = get_spotify_track_id(track_id)
    if spotify_track_id is not None:
        new_row = extract_df_row(spotify_track_id)
        spotify_df.loc[track_id] = new_row

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [173]:
print(spotify_df.head())
print(spotify_df.shape)

                      tempo  time_signature  danceability  energy  key  \
track_id                                                                 
TRAAABD128F429CF47  121.361             4.0         0.748   0.693  9.0   
TRAAADZ128F9348C2E  102.463             4.0         0.699   0.519  8.0   
TRAAAEF128F4273421  119.391             4.0         0.602   0.843  0.0   
TRAAAFD128F92F423A  129.702             4.0         0.527   0.910  2.0   
TRAAAMO128F1481E7F  147.565             4.0         0.456   0.472  5.0   

                    loudness  speechiness  acousticness  instrumentalness  \
track_id                                                                    
TRAAABD128F429CF47    -7.507       0.0302       0.20000          0.000019   
TRAAADZ128F9348C2E    -6.422       0.0290       0.68900          0.000471   
TRAAAEF128F4273421    -9.119       0.0426       0.57500          0.000268   
TRAAAFD128F92F423A    -4.513       0.0582       0.00033          0.657000   
TRAAAMO128F1481E7F 

In [174]:
spotify_df.to_csv(getcwd()+'/spotify_df.csv')